### **Content License Agreement**

<font color='red'><b>**WARNING**</b></font> : 본 자료는 삼성 청년 SW아카데미의 컨텐츠 자산으로, 보안서약서에 의거하여 어떠한 사유로도 임의로 복사, 촬영, 녹음, 복제, 보관, 전송하거나 허가 받지 않은 저장매체를 이용한 보관, 제3자에게 누설, 공개 또는 사용하는 등의 무단 사용 및 불법 배포 시 법적 조치를 받을 수 있습니다.

### **Objectives**

1. 실습명 : 데이터 증강
2. 핵심 주제:
    1. 환경 설정 및 기본 함수 이해
    2. 데이터 '증강'을 위한 Prompt Engineering
    3. 증강 데이터 평가
3. 학습 목표 :
    1. 과제 진행에 필요한 라이브러리와 API 정보를 설정
    2. `Text-to-SQL` task를 위한 학습 데이터를 생성
    3. 생성된 데이터의 품질을 자동으로 평가하는 '평가자(Judge)' LLM을 설계
4. 학습 개념: 키워드명 :
    1. API 통신
    2. Prompt Engineering
    3. LLM as Judge
  
5. 학습 방향 :
  - 실습은 기본적인 API 통신을 통해 데이터를 생성하고, 증강 데이터의 품질을 평가하는 '평가자(Judge)' LLM을 설계하는 과정을 통해 학습됩니다.
  - 실습 코드는 조교가 직접 구현한 코드를 참고하며 학습합니다.
  - 해당 실습에 정답은 없으므로 자신이 옳다고 생각하는 나름의 근거를 세워가며 학습합니다.



### **Prerequisites**


In [ ]:
%pip install dotenv==0.9.9 openai==1.100.0 datasets==4.0.0

# 1. 환경설정

- os environment variable 설정을 진행합니다.


### 1.1. os environment variable

실습 코드에서 처럼 Upstage_API_Key 설정을 위해서 환경변수를 설정합니다.

구글 드라이브에 mount를 합니다.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


mount한 이후 아래 코드를 실행하여 .env 파일을 생성합니다.

API key는 [Upstage Console](https://console.upstage.ai/docs/getting-started)을 확인해주세요.

In [ ]:
# 실습코드를 실행하셨다면 다시 진행하지 않으셔도 됩니다.
your_api_key = "<YOUR_API_KEY>" # 여기에 Upstage API 키를 입력하세요.
!echo "UPSTAGE_API_KEY={your_api_key}" > "/content/drive/My Drive/Colab Notebooks/.env"

저장된 .env 파일을 불러서 환경변수로 설정합니다.

In [ ]:
from dotenv import load_dotenv
from os import getenv
load_dotenv("/content/drive/My Drive/Colab Notebooks/.env")

UPSTAGE_API_KEY = getenv("UPSTAGE_API_KEY")
if UPSTAGE_API_KEY:
    print("Success API Key Setting!")

Success API Key Setting!


이렇게 불러오면 .env 파일을 github에 push할 때, gitignore 처리를 해서 API가 유출되지 않도록 할 수 있습니다.

또한, .env 파일을 암호화해서 보관할 수도 있습니다.

# 2. real data를 이용한 데이터 증강

아무것도 없는 상태에서 합성 데이터를 만드는 것은 굉장히 어렵습니다. 그 이유는 다음과 같습니다.

1. 정답이 없는 상황에서 모델을 이용해서 정답을 만들 경우 현실 세상에 없는 데이터가 만들어지면서 오히려 모델의 성능이 떨어질 수 있습니다.
2. 입력 데이터가 실제 사람으로부터 나온 데이터가 아니기 때문에 실제 입력과의 차이가 발생하여 서비스 단계에서 문제가 발생합니다.

이러한 문제를 어느정도 완화할 수 있는 것이 데이터 증강입니다. 데이터 증강은 real data로부터 조금씩 번형하여 데이터를 만들어내는 것을 의미합니다. 지금부터 데이터를 증강하는 방법을 경험해봅니다.

### 2.1. 영어 데이터 번역

한국어로 된 데이터셋은 많지 않습니다. 그 이유는 다음과 같습니다.

1. 한국어를 사용하는 사람들의 비율이 전세계적으로 적습니다.
2. 한국어로 된 데이터셋을 만드는 기업 or 개인의 수가 적습니다.
3. 한국어로 된 데이터셋을 만들기 위해서는 많은 노동력과 자본이 필요하지만 다른 국가에 비해 비교적 부족합니다.

반대로 영어로된 데이터셋은 많습니다. 영어를 사용하는 인구가 많고, 많은 자본이 동원되어 데이터셋을 만듭니다.

이러한 이유로 영어로 된 공개된 데이터셋을 한국어로 번역하여 학습하는 경우가 많습니다.

지금부터 영어로된 데이터셋을 한국어로 번역하는 방식으로 한국어 데이터를 만들어보겠습니다.

합성 데이터를 만들든 데이터를 증강하든 어떤 데이터셋을 만들든 간에 추가적인 데이터셋을 만들 때에는 다음과 같은 과정으로 진행합니다.

1. 일부 데이터셋으로만 합성 데이터를 제작하는 코드를 만들어서 테스트를 진행합니다.
2. 해당 합성 데이터 제작 코드로 퀄리티가 좋다면 단계적으로 갯수를 늘려서 데이터 퀄리티를 체크합니다. (ex : 10 -> 100 -> 1000)
3. 데이터를 scale-up 해도 퀄리티가 유지가 된다면 전체 데이터셋에 대해서 합성 데이터를 제작합니다.

이러한 방식으로 데이터 생성 비용을 최소화하면서 데이터를 제작하는 파이프라인을 제작합니다.

영어로 된 Text-to-SQL 데이터셋을 불러옵니다.

In [ ]:
from datasets import load_dataset

# 데이터셋 이름
dataset_name = "b-mc2/sql-create-context"

# 데이터셋 Load
dataset = load_dataset(dataset_name, split="train")
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

Dataset({
    features: ['answer', 'question', 'context'],
    num_rows: 78577
})

데이터 생성 파이프라인을 만들기 위해 일부 데이터셋으로만 진행합니다.

In [ ]:
dataset_10 = dataset.select(range(10))
dataset_10

Dataset({
    features: ['answer', 'question', 'context'],
    num_rows: 10
})

데이터셋을 직접 보고 특징을 파악합니다.

In [ ]:
dataset_10[0]

{'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'question': 'How many heads of the departments are older than 56 ?',
 'context': 'CREATE TABLE head (age INTEGER)'}

- answer : SQL 문
- question : 유저의 입력 프롬프트
- context : DB의 table 정보

이 중 한국어 유저는 question 단계(유저 입력 프롬프트)에서 한국어를 사용할 것이므로 `question` 컬럼을 한국어로 번역합니다.

한국어 번역을 하기 위한 `SYSTEM_PROMPT`를 작성합니다.

In [ ]:
# TODO: 다음 영어에서 한국어로 번역하는 지시사항에 관한 시스템 프롬프트를 작성하세요
# SYSTEM_PROMPT = FIXME

안정적인 결과물을 얻기 위해 `response_format`을 정의합니다.

In [ ]:
response_format={
    "type": "json_schema",
    "json_schema": {
        "name": "translation",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "korean": {
                    "type": "string",
                    "description": "translated sentence"
                }
            },
            "required": ["korean"],
        }
    }
}

하나의 입력을 넣어서 제대로 결과물이 나오는지 확인합니다.

In [ ]:
from openai import OpenAI
import json

client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1"
)

system_message = {
    "role": "system",
    "content": SYSTEM_PROMPT
}

for row in dataset_10:
    # TODO : USER MESSAGE를 정의하세요.
    # user_message = FIXME

    messages = [
        system_message,
        user_message
    ]

    response = client.chat.completions.create(
        model="solar-pro2",
        messages=messages,
        response_format=response_format
    )

    output = response.choices[0].message.content
    print(type(output))
    print(output)

    json_output = json.loads(output)

    print(type(json_output))
    print(json_output)
    break

<class 'str'>
{
  "korean": "56세 이상인 부서장들은 몇 명인가요?"
}
<class 'dict'>
{'korean': '56세 이상인 부서장들은 몇 명인가요?'}


# 2.2. 데이터 증강

영어 데이터로부터 다양한 스타일의 한국어로 번역하는 과정을 통해 다양한 데이터를 얻을 수 있습니다.

실제 모델을 서비스를 한다면 초등학생, 직장인, 주부 등 다양한 유저들이 모델을 사용할 것입니다. 이러한 다양한 상황들을 전부 학습하는 것이 모델의 성능을 높이고 안정적인 서비스를 유지하는데 핵심입니다.

다양한 번역 결과물을 얻기 위해서 다양한 페르소나를 정의합니다.

In [ ]:
# TODO: 초등학생, 개발자, PM 등 다양한 페르소나를 정의하세요
# personas = [] FIXME

SYSTEM_PROMPT_WITH_PERSONA = """
You are a translation assistant.
Your task is to translate English sentences into Korean.
You must keep the meaning accurate, but adapt the tone and style according to the requested persona.

{persona}
"""

translated_questions_with_persona = []
for idx, row in enumerate(dataset_10):
    output_row = {
        "id": idx,
        "question_with_persona": []
    }
    for persona in personas:
        system_message = {
            "role": "system",
            "content": SYSTEM_PROMPT_WITH_PERSONA.format(persona=persona)
        }
        user_message = {
            "role": "user",
            "content": row["question"]
        }

        messages = [
            system_message,
            user_message
        ]

        response = client.chat.completions.create(
            model="solar-pro2",
            messages=messages,
            response_format=response_format
        )

        output = response.choices[0].message.content
        json_output = json.loads(output)
        output_row["question_with_persona"].append({
            "persona": persona,
            "question_korean": json_output["korean"]
        })

    translated_questions_with_persona.append(output_row)
    break
translated_questions_with_persona

[{'id': 0,
  'question_with_persona': [{'persona': 'Friendly Adult – Kind and polite tone, like an older sibling giving advice.',
    'question_korean': '56세보다 연장자이신 부서장님들께서는 몇 분 정도 되시나요?'},
   {'persona': 'Elementary Student – Simple, cheerful, and naive tone, like a 10-year-old child.',
    'question_korean': '56살보다 나이 많은 부서장들이 몇 명이에요?'},
   {'persona': 'Formal Business – Polite, professional tone suitable for documents or work emails.',
    'question_korean': '56세 이상인 부서장 수는 몇 명입니까?'},
   {'persona': 'Casual Friend – Relaxed, informal tone, as if chatting with a close friend.',
    'question_korean': '부서장들 중 56살 넘은 사람 몇 명이나 되지?'},
   {'persona': 'Elderly Person – Gentle, old-fashioned, and wise tone, like a kind grandparent.',
    'question_korean': '56세보다 연세가 많으신 부서장님들은 몇 분이나 되시나요?'}]}]

# 3. 증강 데이터 평가

합성 데이터 혹은 증강 데이터를 평가할 때에는 정성적인 부분이 많이 들어갑니다. 왜냐하면 정상적인 데이터가 아닐 수 있기 때문에 명확히 평가하기 어려운 부분도 있습니다. 평가가 어려운 이유는 다음과 같습니다.

-  모델의 강건함(robustness)를 올리기 위해 일부러 비정상적인 데이터를 넣을 수 있습니다. 이 경우 명확한 평가가 어렵습니다.
-  다양한 데이터를 얻기 위해 스타일을 바꾼 것은 명확한 정량적인 평가가 어렵습니다.

또한, 모델 학습 데이터를 얻기 위해서 데이터를 증강했다는 의미는 **모델의 성능이 올랐냐**가 가장 중요한 평가 기준이 됩니다. 따라서, task, 모델 등에 따라서 모델의 성능이 각기 달라지기 때문에 명확한 정답이 있는 것은 아닙니다.

잘 만들었다는 증강 데이터는 모델의 학습과 연관이 되어 있기 때문에 모델 학습 결과를 보면서 ML 엔지니어의 감과 노하우가 들어가는 영역입니다.

즉, 해당 task의 평가 데이터 혹은 실제 서비스상에서 나타나는 데이터들을 보고 그것과 유사한 증강 데이터를 만드는 등 좀더 실무적인 영역이 많이 들어가기 때문에 평가하기가 더더욱 어렵습니다.

우선 간단하게 해볼 수 있는 평가는 LLM as a Judge를 이용하여 GPT를 통해 한번 필터링을 하는 것입니다. 그 이후에는 모델 영역으로도 평가하기 어려운 부분은 직접 ML 엔지니어가 판단할 수 있어야 합니다.


그러면 LLM as a Judge를 이용하여 이전에 생성한 합성 데이터 결과물을 평가해보겠습니다.

In [ ]:
# TODO: 원본 영어 텍스트의 의미를 훼손하지 않으면서 다양한 한국어 데이터를 얻었는지 판단하기 위한 SYSTEM PROMPT를 작성합니다.
# JUDGE_SYSTEM_PROMPT= FIXME

USER_PROMPT = """- original sentence : {english}
- translated sentence : {korean}
"""

# TODO: system prompt에서 output으로 설정할 response_format을 정의하세요. "score"와 "reason"이 키값으로 들어가야 합니다.
# evaluate_response_format= FIXME

system_message = {
    "role": "system",
    "content": JUDGE_SYSTEM_PROMPT
}


evaluated_questions_with_persona = []
for row, translated_question_with_persona in zip(dataset_10, translated_questions_with_persona):

    for idx, question_with_persona in enumerate(translated_question_with_persona["question_with_persona"]):
        user_message = {
            "role": "user",
            "content": USER_PROMPT.format(
                english=row["question"],
                korean=question_with_persona["question_korean"]
            )
        }

        messages = [
            system_message,
            user_message
        ]

        response = client.chat.completions.create(
            model="solar-pro2",
            messages=messages,
            response_format=evaluate_response_format
        )

        output = response.choices[0].message.content
        json_output = json.loads(output)
        evaluated_question_with_persona = translated_question_with_persona.copy()
        evaluated_question_with_persona["question_with_persona"][idx]["score"] = json_output["score"]
        evaluated_question_with_persona["question_with_persona"][idx]["reason"] = json_output["reason"]
        evaluated_questions_with_persona.append(evaluated_question_with_persona)
    break
evaluated_questions_with_persona

[{'id': 0,
  'question_with_persona': [{'persona': 'Friendly Adult – Kind and polite tone, like an older sibling giving advice.',
    'question_korean': '56세보다 연장자이신 부서장님들께서는 몇 분 정도 되시나요?',
    'score': 8,
    'reason': "The Korean translation preserves the core meaning of asking about the number of department heads older than 56. However, the translation adds honorifics ('연장자이신', '님들', '되시나요') that change the tone to very formal/polite, which wasn't explicitly present in the original neutral English question. No factual information is distorted or omitted, but the elevated politeness level constitutes a tone/persona shift rather than just a neutral translation."},
   {'persona': 'Elementary Student – Simple, cheerful, and naive tone, like a 10-year-old child.',
    'question_korean': '56살보다 나이 많은 부서장들이 몇 명이에요?',
    'score': 9,
    'reason': "The Korean translation accurately conveys the original meaning by asking about the number of department heads older than 56. The only minor diff

평가한 데이터 중에서 몇점 이상의 데이터만 남기고 나머지는 필터링할지 정합니다.

In [ ]:
# TODO: 필터링할 스코어를 설정해주세요
# filtering_score = FIXME

synthetic_datasets = []
for row, evaluated_question_with_persona in zip(dataset_10, evaluated_questions_with_persona):
    for question_with_persona in evaluated_question_with_persona["question_with_persona"]:
        if question_with_persona["score"] < filtering_score:
            continue

        synthetic_row = {
            "question": question_with_persona["question_korean"],
            "context": row["context"],
            "answer": row["answer"]
        }
        synthetic_datasets.append(synthetic_row)

    break

synthetic_datasets

[{'question': '56살보다 나이 많은 부서장들이 몇 명이에요?',
  'context': 'CREATE TABLE head (age INTEGER)',
  'answer': 'SELECT COUNT(*) FROM head WHERE age > 56'},
 {'question': '56세 이상인 부서장 수는 몇 명입니까?',
  'context': 'CREATE TABLE head (age INTEGER)',
  'answer': 'SELECT COUNT(*) FROM head WHERE age > 56'},
 {'question': '부서장들 중 56살 넘은 사람 몇 명이나 되지?',
  'context': 'CREATE TABLE head (age INTEGER)',
  'answer': 'SELECT COUNT(*) FROM head WHERE age > 56'},
 {'question': '56세보다 연세가 많으신 부서장님들은 몇 분이나 되시나요?',
  'context': 'CREATE TABLE head (age INTEGER)',
  'answer': 'SELECT COUNT(*) FROM head WHERE age > 56'}]

지금까지 데이터를 증강하기 위한 기본 환경 설정과 프롬프트 엔지니어링, LLM as a Judge 방식을 배워보았습니다.

데이터를 증강하는 방식에 정답은 없습니다. 하지만, 가장 중요한 것은 real data와 가장 비슷하게 만들어내는 것입니다.

real data와 비슷하게 만든다는 것은 근본적으로 내가 만들고자 하는 서비스(만들고자 하는 모델)이 무엇이냐에 따라서 달라집니다.

따라서, 문제 정의(데이터의 목적)에 따라 다양하게 데이터를 증강하는 것을 시도해보시길 바랍니다.